In [1]:
import pandas as pd,numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score,cohen_kappa_score,confusion_matrix,recall_score,precision_score
from sklearn.linear_model import LogisticRegression
import plotly.express as px
from datetime import datetime
import time, random
import xgboost as xgb
#tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
train = pd.read_csv("https://raw.githubusercontent.com/Layden23/bet369/master/Datasets/train2.csv", header = 0)

In [3]:
len(train["MatchID"].unique())

1679

In [4]:
train.isna().sum()

MatchID                    18
LeagueID                   18
LeagueName                 18
LeagueFullname             18
LeagueAbbreviation         18
CountryID                  18
CountryName                18
CountryAbbreviation        18
HomeTeamID                 18
HomeTeamName               18
GuestTeamID                18
GuestTeamName              18
HostHandicapFT             75
ExpectedGoalsFT            77
BettingCuoteFT         151720
HostHandicapHT            274
ExpectedGoalsHT           268
BettingCuoteHT         151870
HostGoalsFT                18
GuestGoalsFT               18
HostCornersFT              18
GuestCornersFT             18
HostYCFT                   18
GuestYCFT                  18
HostRCFT                   18
GuestRCFT                  18
HostAttacksFT              18
GuestAttacksFT             18
HostDAttacksFT             18
GuestDAttacksFT            18
HostonTargetFT             18
GuestonTargetFT            18
HostoffTargetFT            18
GuestoffTa

In [5]:
train = train.drop(train[train["MatchID"] == "MatchID"].index)
train = train.drop_duplicates(subset="MatchID")

In [6]:
#Remove Strange Matches
train = train.loc[(train['HostPossessionFT'] != "0") & (train['GuestPossessionFT'] != "0")]
#Fix Goal Problem
train["Goal"] = pd.to_numeric(train["HostGoalsFT"]) + pd.to_numeric(train["GuestGoalsFT"])

In [7]:
s = 69

In [8]:
#Remove columns with lots of missing values
del train["BettingCuoteHT"]
del train["BettingCuoteFT"]
del train["HostGoalsFT"]
del train["GuestGoalsFT"]

In [9]:
#Creating train with matches that score in the first half
# train.Status = train.Status.replace(["HT","FT"],[45,90])
train = train.drop(train[train["Status"] == "NS"].index)
train = train.drop(train[train["Status"] == "HT"].index)
train = train.drop(train[train["Status"] == "FT"].index)
#Transform variables to numeric
for i in train.iloc[:,12:].columns:
    train[i] = pd.to_numeric(train[i])
#Creating train with matches that don't score in the first half
train0a = train[(train["Goal"] == 0) & (train["Status"] <= 45)]
train0 = train0a.groupby('MatchID').apply(lambda x: x.sample(1))

In [10]:
train0

,,MatchID,LeagueID,LeagueName,LeagueFullname,LeagueAbbreviation,CountryID,CountryName,CountryAbbreviation,HomeTeamID,HomeTeamName,...,HostDAttacksFT,GuestDAttacksFT,HostonTargetFT,GuestonTargetFT,HostoffTargetFT,GuestoffTargetFT,HostPossessionFT,GuestPossessionFT,Status,Goal
MatchID,,,,,,,,,,,,,,,,,,,,,,
682951,60434,682951,39,FRA L1,France Ligue 1,F,31,France,F,1098,Brest,...,18.0,20.0,1.0,0.0,4.0,3.0,47.0,53.0,36.0,0.0
682953,60444,682953,39,FRA L1,France Ligue 1,F,31,France,F,1054,Montpellier,...,24.0,23.0,2.0,2.0,4.0,5.0,53.0,47.0,36.0,0.0
682954,60441,682954,39,FRA L1,France Ligue 1,F,31,France,F,721,Reims,...,7.0,6.0,0.0,1.0,0.0,3.0,51.0,49.0,36.0,0.0
683472,60457,683472,1813,BEL FDA,Belgium First Division A,B,38,Belgium,B,5523,Genk,...,8.0,0.0,1.0,0.0,0.0,0.0,56.0,44.0,6.0,0.0
683550,60437,683550,1813,BEL FDA,Belgium First Division A,B,38,Belgium,B,1122,Zulte-Waregem,...,15.0,22.0,0.0,1.0,2.0,1.0,53.0,47.0,36.0,0.0
683566,60439,683566,1813,BEL FDA,Belgium First Division A,B,38,Belgium,B,5927,Antwerp,...,29.0,32.0,3.0,1.0,3.0,2.0,55.0,45.0,36.0,0.0
683586,4174,683586,32,SAU PL,Saudi Arabia Premier League,S,36,Saudi Arabia,S,18792,Dhamk,...,10.0,5.0,2.0,0.0,0.0,1.0,51.0,49.0,9.0,0.0
684114,7412,684114,3,AUS AL,Australia A-League,A,7,Australia,A,334,Western Sydney Wanderers,...,7.0,11.0,1.0,0.0,0.0,1.0,35.0,65.0,12.0,0.0
684245,7296,684245,214,BRA CB SB,Campeonato Brasileiro Serie B,C,47,Brazil,B,5876,Londrina,...,13.0,18.0,0.0,1.0,1.0,1.0,39.0,61.0,24.0,0.0


In [11]:
len(train0)

60

In [12]:
train1 = train[(train['Status'] <= 45) & (train["Goal"]==1)]
train1 = train1.sort_values("Status").groupby("MatchID", as_index=False).first()

In [13]:
train1.sort_values("Goal", ascending = False).iloc[:,-20:]

,HostHandicapHT,ExpectedGoalsHT,HostCornersFT,GuestCornersFT,HostYCFT,GuestYCFT,HostRCFT,GuestRCFT,HostAttacksFT,GuestAttacksFT,HostDAttacksFT,GuestDAttacksFT,HostonTargetFT,GuestonTargetFT,HostoffTargetFT,GuestoffTargetFT,HostPossessionFT,GuestPossessionFT,Status,Goal
0,-0.25,0.75,2.0,3.0,1.0,0.0,0.0,0.0,53.0,33.0,23.0,14.0,3.0,2.0,5.0,2.0,57.0,43.0,36.0,1.0
1,0.00,1.00,1.0,2.0,1.0,0.0,0.0,0.0,28.0,29.0,14.0,11.0,2.0,2.0,1.0,2.0,42.0,58.0,30.0,1.0
2,0.00,1.25,0.0,1.0,0.0,0.0,0.0,0.0,30.0,26.0,4.0,7.0,1.0,0.0,0.0,0.0,51.0,49.0,20.0,1.0
3,0.00,0.75,2.0,1.0,0.0,1.0,0.0,0.0,25.0,31.0,26.0,16.0,3.0,2.0,1.0,3.0,54.0,46.0,26.0,1.0
4,0.50,1.00,1.0,5.0,0.0,0.0,0.0,0.0,41.0,57.0,18.0,26.0,2.0,3.0,3.0,2.0,39.0,61.0,45.0,1.0
5,0.25,1.00,1.0,1.0,0.0,0.0,0.0,0.0,34.0,26.0,18.0,14.0,2.0,2.0,0.0,4.0,60.0,40.0,30.0,1.0
6,-1.00,1.25,2.0,0.0,0.0,0.0,0.0,0.0,38.0,41.0,37.0,24.0,3.0,2.0,5.0,0.0,48.0,52.0,31.0,1.0
7,-0.50,1.00,1.0,3.0,0.0,2.0,0.0,0.0,29.0,25.0,11.0,15.0,2.0,2.0,0.0,1.0,57.0,43.0,36.0,1.0
8,0.00,0.75,2.0,1.0,0.0,1.0,0.0,0.0,51.0,42.0,31.0,26.0,0.0,1.0,4.0,0.0,54.0,46.0,38.0,1.0
9,-0.25,1.00,2.0,6.0,0.0,0.0,0.0,0.0,39.0,34.0,20.0,24.0,0.0,1.0,5.0,4.0,47.0,53.0,37.0,1.0


In [14]:
len(train1)

13

In [15]:
len(train1.MatchID.unique())

13

In [16]:
ntrain = train1.append(train0)


In [17]:
ntrain

,MatchID,LeagueID,LeagueName,LeagueFullname,LeagueAbbreviation,CountryID,CountryName,CountryAbbreviation,HomeTeamID,HomeTeamName,...,HostDAttacksFT,GuestDAttacksFT,HostonTargetFT,GuestonTargetFT,HostoffTargetFT,GuestoffTargetFT,HostPossessionFT,GuestPossessionFT,Status,Goal
0,682955,39,FRA L1,France Ligue 1,F,31,France,F,2341,Strasbourg,...,23.0,14.0,3.0,2.0,5.0,2.0,57.0,43.0,36.0,1.0
1,683585,32,SAU PL,Saudi Arabia Premier League,S,36,Saudi Arabia,S,8010,Al Feiha,...,14.0,11.0,2.0,2.0,1.0,2.0,42.0,58.0,30.0,1.0
2,684165,40,NED E,Holland Eredivisie,H,25,the Netherlands,H,6788,Fortuna Sittard,...,4.0,7.0,1.0,0.0,0.0,0.0,51.0,49.0,20.0,1.0
3,684244,214,BRA CB SB,Campeonato Brasileiro Serie B,C,47,Brazil,B,1480,Atletico GO,...,26.0,16.0,3.0,2.0,1.0,3.0,54.0,46.0,26.0,1.0
4,684747,151,ISR PL,Israel Premier League,I,97,Israel,Y,5919,Hapoel Kfar Saba,...,18.0,26.0,2.0,3.0,3.0,2.0,39.0,61.0,45.0,1.0
5,686008,2181,ROM L2,Romania Liga II,R,30,Romania,L,1415,Pandurii Targu Jiu,...,18.0,14.0,2.0,2.0,0.0,4.0,60.0,40.0,30.0,1.0
6,686014,289,MDA DN,Moldova Divizia Nationala,M,58,Moldova,M,8911,CS Petrocub,...,37.0,24.0,3.0,2.0,5.0,0.0,48.0,52.0,31.0,1.0
7,686379,753,ESP TG10,Spain Tercera Group 10,S,23,Spain,X,4672,Real Betis B,...,11.0,15.0,2.0,2.0,0.0,1.0,57.0,43.0,36.0,1.0
8,686618,1690,Mali PD,Mali Premiere Division,M,165,Mali,M,21548,AS Olympique de Missira,...,31.0,26.0,0.0,1.0,4.0,0.0,54.0,46.0,38.0,1.0
9,686903,287,BRA U20,Brazil U20 League,B,47,Brazil,B,23832,Salgueiro PE U20,...,20.0,24.0,0.0,1.0,5.0,4.0,47.0,53.0,37.0,1.0


In [18]:
ntrain.isna().sum()

MatchID                0
LeagueID               0
LeagueName             0
LeagueFullname         0
LeagueAbbreviation     0
CountryID              0
CountryName            0
CountryAbbreviation    0
HomeTeamID             0
HomeTeamName           0
GuestTeamID            0
GuestTeamName          0
HostHandicapFT         1
ExpectedGoalsFT        1
HostHandicapHT         2
ExpectedGoalsHT        2
HostCornersFT          0
GuestCornersFT         0
HostYCFT               0
GuestYCFT              0
HostRCFT               0
GuestRCFT              0
HostAttacksFT          0
GuestAttacksFT         0
HostDAttacksFT         0
GuestDAttacksFT        0
HostonTargetFT         0
GuestonTargetFT        0
HostoffTargetFT        0
GuestoffTargetFT       0
HostPossessionFT       0
GuestPossessionFT      0
Status                 0
Goal                   0
dtype: int64

In [19]:
ntrain = ntrain.dropna()
ntrain = ntrain.drop_duplicates(subset="MatchID")

In [20]:
len(ntrain)

71

In [21]:
# Indicies of each class' observations
random.seed(s)
i_class0 = np.where(ntrain["Goal"] == 0)[0]
i_class1 = np.where(ntrain["Goal"] == 1)[0]

#Get size of the underrepresented class
n_class0 = len(i_class0)

i_class1_downsampled = np.random.choice(i_class1, size=n_class0, replace=False)
ntrain = ntrain.iloc[i_class0,:].append(ntrain.iloc[i_class1_downsampled,:])

ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:
ntrain.sort_values(by = "Goal", ascending = False)

In [ ]:
px.histogram(ntrain[ntrain["Goal"]== 0], x = "Status", nbins = 45)

In [ ]:
px.histogram(ntrain[ntrain["Goal"]== 1], x = "Status", nbins = 45)

In [ ]:
px.histogram(ntrain, x = "Goal")

# Feature Engineering

In [ ]:
X = ntrain.iloc[:,12:32]
stand = MinMaxScaler().fit(X)
X = pd.DataFrame(stand.transform(X))
pca = PCA(0.99)
pca.fit(X)
pcX = pca.transform(X)
y = ntrain["Goal"]
#Transform variables to numeric
for i in X.columns:
    X[i] = pd.to_numeric(X[i])
y = pd.to_numeric(y)



In [ ]:
random.seed(s)
X_train, X_test, y_train, y_test = train_test_split(pcX,y,train_size = 0.7)

In [ ]:
#Random Forest
random.seed(s)
rf = RandomForestClassifier(n_estimators = 200)
rf.fit(X_train, y_train)


In [ ]:
# Logistic Regression
random.seed(s)
lr = LogisticRegression()
lr.fit(X_train, y_train)

In [ ]:
#XGBTree 
random.seed(s)
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic'}
xgb = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'accuracy', cv = 5, n_jobs = -1)
xgb.fit(X_train,y_train)

In [ ]:
#Neuronal network
tf.keras.backend.clear_session() # For easy reset of notebook state.
#### Ignacio: You MUST explain a bit your neural network topology: 
#### One hidden layer with 128 neurons with the "Relux" activation function 
#### which is better for computational efficiency. Then you add an output layer with 3 neurons (one for each building you want to predict).
#### As the output neurons give you some value between 0-1, you apply the softmax function in order to normalize those outputs and interpret them 
#### as probabilities of being in each building.
nmodel = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(2, activation=tf.nn.softmax)
])

#### Ignacio: Why this choices of optimizer and loss function and metrics?
nmodel.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
nmodel.fit(np.array(X_train), np.array(y_train), epochs=10)

In [ ]:
test_loss, test_acc = nmodel.evaluate(X_test, y_test, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
#Predictions
rfpreds = rf.predict(X_test)
lrpreds = lr.predict(X_test)
xgbpreds = xgb.predict(X_test)
npreds = np.round(nmodel.predict(X_test)[:,1])

In [ ]:
#Probabilities
rfprobs = rf.predict_proba(X_test)[:,1]
lrprobs = lr.predict_proba(X_test)[:,1]
xgbprobs = xgb.predict_proba(X_test)[:,1]
nprobs = nmodel.predict(X_test)[:,1]

In [ ]:
importances = rf.feature_importances_
cols = X.columns
pd.DataFrame({"Variable": cols,"Importance" : importances})

In [ ]:
results = pd.DataFrame({"Goal": y_test,"rfpreds" : rfpreds,"lrpreds": lrpreds, "xgbpreds": xgbpreds,
              "rfprobs": rfprobs, "lrprobs": lrprobs, "xgbprobs": xgbprobs, "npreds": npreds, "nprobs": nprobs})

In [ ]:
results

In [ ]:
rf_test = results[results["rfprobs"] >= 0.9]
print(accuracy_score(rf_test["Goal"],rf_test["rfpreds"]))
print(accuracy_score(rfpreds,y_test))

In [ ]:
rf_test

In [ ]:
lr_test = results[results["lrprobs"] >= 0.9]
print(accuracy_score(lr_test["Goal"],lr_test["lrpreds"]))
print(accuracy_score(lrpreds,y_test))

In [ ]:
lr_test

In [ ]:
xgb_test = results[results["xgbprobs"] >= 0.9]
print(accuracy_score(xgb_test["Goal"],xgb_test["xgbpreds"]))
print(accuracy_score(xgbpreds,y_test))

In [ ]:
xgb_test

In [ ]:
n_test = results[results["nprobs"] >= 0.9]
print(accuracy_score(n_test["Goal"],n_test["npreds"]))
print(accuracy_score(npreds,y_test))

In [ ]:
n_test

In [ ]:
confusion_matrix(xgbpreds,y_test)

In [ ]:
confusion_matrix(lrpreds,y_test)


In [ ]:
confusion_matrix(rfpreds,y_test)

In [ ]:
confusion_matrix(npreds,y_test)

In [ ]:
print("Metric   "," RF "," LR ","XGB","NN")
print("Accuracy ", round(accuracy_score(rfpreds,y_test),2), round(accuracy_score(lrpreds,y_test),2), round(accuracy_score(xgbpreds,y_test),2),round(accuracy_score(npreds, y_test),2))
print("Kappa    ",round(cohen_kappa_score(rfpreds,y_test),2),round(cohen_kappa_score(lrpreds,y_test),2),round(cohen_kappa_score(xgbpreds,y_test),2),round(cohen_kappa_score(npreds, y_test),2))
print("Precision",round(precision_score(rfpreds,y_test),2),round(precision_score(lrpreds,y_test),2),round(precision_score(xgbpreds,y_test),2),round(precision_score(npreds, y_test),2))
print("Recall   ", round(recall_score(rfpreds,y_test),2),round(recall_score(lrpreds,y_test),2),round(recall_score(xgbpreds,y_test),2),round(recall_score(npreds, y_test),2))

# Predictions Live

In [ ]:
filename = datetime.now().strftime('%Y-%m-%d/gambling-%Y-%m-%d-%H-%M.csv')
test = pd.read_csv(filename, header = 0)
del test["BettingCuoteHT"]
del test["BettingCuoteFT"]
del test["HostGoalsFT"]
del test["GuestGoalsFT"]
test = test.dropna()
test = test.loc[(test['HostPossessionFT'] != 0) & (test['GuestPossessionFT'] != 0)]
test = test.drop(test[test["Status"] == "HT"].index)
test = test.drop(test[test["Status"] == "NS"].index)
test = test.drop(test[test["Status"] == "FT"].index)
ntest = test.iloc[:,12:32]
pctest = pca.transform(ntest)
tpreds = rf.predict(pctest)
tprobs = rf.predict_proba(pctest)[:,1]
test["Prediction"] = (np.round(xgb.predict(pctest)))
test["Probability"] = rf.predict_proba(pctest)[:,1]
test["Status"] = pd.to_numeric(test["Status"])
test0 = test[(test["Status"] <= 45) & (test["Goal"] == 0)]
test0 = test0[["HomeTeamName","GuestTeamName","Status","Goal","Prediction","Probability","HostonTargetFT","GuestonTargetFT","HostCornersFT","GuestCornersFT",
       "HostYCFT","GuestYCFT","HostRCFT",
      "GuestRCFT","HostAttacksFT","GuestAttacksFT","HostoffTargetFT",
      "GuestoffTargetFT","HostPossessionFT","GuestPossessionFT"]]

test0.sort_values("Probability", ascending = False)


In [ ]:
#Independent of time/Status
#Class balanced
#Less observations